In [1]:
!pip install sentencepiece


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from __future__ import annotations  # 미래 버전의 기능을 미리 사용 가능하도록 설정
import os  # 운영체제 기능 사용 (파일 관련 등)
import sys  # 파이썬 인터프리터 관련 기능 사용
import subprocess  # 다른 프로그램을 파이썬에서 실행할 때 사용
import math  # 수학적 계산을 위한 라이브러리 (삼각함수, 제곱근 등)
import random  # 랜덤 값 생성에 사용 (무작위 수 생성)
import re  # 정규표현식 사용 (텍스트 처리에 활용)
import inspect  # 객체 검사 및 정보를 얻기 위한 라이브러리
from dataclasses import dataclass  # 데이터 클래스를 간편히 생성하기 위한 기능
from pathlib import Path  # 파일 경로를 객체지향적으로 다룰 수 있게 해주는 라이브러리
from typing import List, Tuple  # 타입 힌트(자료형 명시)용
import sentencepiece as spm  # 텍스트를 단어 또는 토큰으로 쪼개주는 라이브러리
import pandas as pd  # 데이터를 표 형식으로 다룰 때 사용하는 라이브러리
import torch  # 인공지능 모델 학습과 연산에 사용하는 주요 프레임워크
import torch.nn as nn  # 신경망(Neural Network)을 만들 때 사용하는 모듈
import torch.nn.functional as F  # 신경망용 함수(예: 활성화 함수) 제공
from torch.utils.data import DataLoader, Dataset  # 데이터를 묶어서 모델 학습에 넣을 수 있게 준비해주는 도구
from sklearn.model_selection import train_test_split  # 데이터를 훈련용과 검증용으로 나누는 함수
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# pack_padded_sequence 함수는 길이가 서로 다른 여러 문장을 한 번에 LSTM 같은 신경망에 입력할 때 사용합니다.
# 예를 들어 ["안녕", "안녕하세요", "안녕하세요 여러분"] 문장을 숫자 벡터로 바꿔서 신경망에 넣을 때,
# 길이가 달라 빈 공간을 채우는 패딩이 필요합니다.
# pack_padded_sequence를 사용하면 신경망이 이 패딩된 공간을 무시하고 실제 길이만큼만 연산하여 효율성을 높입니다.
# pad_packed_sequence 함수는 위에서 pack_padded_sequence로 압축된 데이터를 다시 원래의 패딩된 형태로 복구할 때 사용합니다.


# 난수 생성 시 결과를 항상 동일하게 유지하기 위한 설정 (재현성 보장)
seed = 42
random.seed(seed)               # 파이썬 random 모듈의 난수 고정
torch.manual_seed(seed)         # 토치의 CPU 난수 고정
torch.cuda.manual_seed_all(seed) # 토치의 GPU 난수 고정
torch.backends.cudnn.deterministic = True   # 쿠다(CUDA) 연산 시 결정적 결과 보장 (항상 동일한 결과가 나오도록)
torch.backends.cudnn.benchmark = False      # 성능 최적화 옵션을 끄고 재현성 보장 (항상 같은 결과 유지)

# 사용할 장치 선택 (GPU 사용 가능하면 GPU 사용, 없으면 CPU 사용)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cuda')

In [3]:
# 한 번에 인공지능 모델이 학습할 데이터의 개수를 설정합니다.
# 예를 들어 BATCH_SIZE = 64이면, 한 번의 학습 시 데이터를 64개씩 가져와서 학습합니다.
BATCH_SIZE = 64


In [4]:
# 텍스트에서 필요 없는 문자들을 제거하여 깨끗하게 만드는 함수입니다.
# 예를 들어, "안녕하세요!!! 저는 AI입니다.^^;" 같은 문장에서 특수문자를 모두 제거하면
# "안녕하세요 저는 AI입니다"로 바뀌게 됩니다.
def clean_text(s: str) -> str:
    # 한글, 영어, 숫자, 공백을 제외한 모든 특수문자 제거합니다.
    # 예: "안녕!! 잘 지내니?" → "안녕 잘 지내니"
    s = re.sub(r"[^가-힣a-zA-Z0-9 ]", "", s)

    # 연속된 여러 개의 공백을 하나의 공백으로 바꾸고, 양 끝의 공백을 없앱니다.
    # 예: "안녕   잘   지내니 " → "안녕 잘 지내니"
    s = re.sub(r"\s+", " ", s).strip()

    # 정리된 텍스트를 반환합니다.
    return s


# CSV 파일에서 챗봇 학습을 위한 데이터를 읽어옵니다.
# 예를 들어 'ChatbotData.csv'에는 질문(Q)과 대답(A)이 들어있습니다.
df = pd.read_csv("ChatbotData.csv", encoding="utf-8-sig")

# 불러온 데이터를 화면에 출력하여 제대로 불러왔는지 확인합니다.
df


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [5]:
# 질문(Q) 데이터를 모두 문자열로 변환 후 깨끗한 텍스트로 정리합니다.
df["Q"] = df["Q"].astype(str).map(clean_text)
# df["Q"]는 질문 열을 나타냅니다.
# astype(str)는 질문 데이터를 모두 문자열 타입으로 변경합니다. 예: 숫자 123은 문자열 "123"으로 변환됩니다.
# map(clean_text)는 질문 데이터 각각에 clean_text 함수를 적용해 특수문자와 공백을 정리합니다.
# 예를 들어 질문이 "안녕!!   반가워요~" 라면,
# clean_text를 적용한 후 "안녕 반가워요"로 바뀌어 저장됩니다.

# df 데이터프레임을 출력해서 질문 데이터가 잘 정리되었는지 화면에서 확인합니다.
df

,Q,A,label
0,12시 땡,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [6]:
df["A"] = df["A"].astype(str).map(clean_text)
# df["A"]는 답변 열을 나타냅니다.
# astype(str)는 답변 데이터를 모두 문자열 타입으로 변경합니다. 예: 숫자 456은 문자열 "456"으로 변환됩니다.
# map(clean_text)는 답변 데이터 각각에 clean_text 함수를 적용해 특수문자와 공백을 정리합니다.
# 예를 들어 답변이 "네!!! 좋아요~"라면,
# clean_text를 적용한 후 "네 좋아요"로 바뀌어 저장됩니다.

# df 데이터프레임을 출력해서 답변 데이터가 잘 정리되었는지 화면에서 확인합니다.
df


,Q,A,label
0,12시 땡,하루가 또 가네요,0
1,1지망 학교 떨어졌어,위로해 드립니다,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,0
4,PPL 심하네,눈살이 찌푸려지죠,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임,티가 나니까 눈치가 보이는 거죠,2
11819,훔쳐보는 것도 눈치 보임,훔쳐보는 거 티나나봐요,2
11820,흑기사 해주는 짝남,설렜겠어요,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까,잘 헤어질 수 있는 사이 여부인 거 같아요,2


In [7]:
# 아래는 특수한 단어(토큰)를 정의하는 부분입니다.
PAD, SOS, EOS, UNK = "<pad>", "<sos>", "<eos>", "<unk>"

# <pad>: 문장의 길이가 서로 다를 때, 짧은 문장 뒤에 길이를 맞추기 위해 빈 공간을 채우는 용도로 쓰입니다.
# 예를 들어, ["안녕", "안녕하세요"] 두 문장을 최대 길이(5자)로 맞출 때,
# ["안녕<pad><pad><pad>", "안녕하세요"]와 같이 빈 공간을 채웁니다.

# <sos>: 문장의 시작을 나타내는 특수 토큰입니다. "start of sentence"의 줄임말입니다.
# 예를 들어, "좋은 하루야"라는 문장을 표현할 때 ["<sos> 좋은 하루야"]처럼 문장 맨 앞에 붙여 시작을 나타냅니다.

# <eos>: 문장의 끝을 나타내는 특수 토큰입니다. "end of sentence"의 줄임말입니다.
# 예를 들어, "좋은 하루야"라는 문장을 표현할 때 ["좋은 하루야 <eos>"]처럼 문장 맨 뒤에 붙여 끝을 나타냅니다.

# <unk>: 학습 데이터에 없는, 처음 보는 단어가 나타났을 때 사용하는 특수 토큰입니다. "unknown"의 줄임말입니다.
# 예를 들어, 학습 시 "핸드폰"이라는 단어를 보지 못했다면, 실제 사용 시 "핸드폰"이 나타날 때 "<unk>"로 처리합니다.

# 센텐스피스(SentencePiece) 모델의 파일명 앞부분을 정의합니다.
# 센텐스피스는 문장을 단어보다 작은 토큰으로 나누는 역할을 합니다.
SPM_MODEL_PREFIX = "spm_ko_nopunct"
# 위 변수는 센텐스피스 모델 파일의 앞부분 이름입니다.
# 예를 들어 실제 파일은 "spm_ko_nopunct.model"과 같은 형태로 생성됩니다.

# 센텐스피스 모델 파일의 전체 이름을 정의합니다.
SPM_MODEL = f"{SPM_MODEL_PREFIX}.model"
# f"{SPM_MODEL_PREFIX}.model"은 문자열 앞에 정의된 SPM_MODEL_PREFIX를 이용해 자동으로 문자열을 구성합니다.
# 예: SPM_MODEL_PREFIX가 "spm_ko_nopunct"일 때, 이 변수는 "spm_ko_nopunct.model"이 됩니다.

# 센텐스피스의 어휘 크기(단어의 수)를 정의합니다.
SPM_VOCAB_SIZE = 8000
# 위 숫자는 센텐스피스가 생성할 수 있는 서로 다른 토큰(작은 단어조각)의 최대 수입니다.
# 숫자가 클수록 다양한 단어를 더 잘 표현할 수 있지만, 너무 크면 학습이 어렵고
# 너무 작으면 단어를 충분히 표현하지 못할 수 있습니다.

# 센텐스피스가 학습할 때 포함할 문자(글자)의 비율을 정의합니다.
CHAR_COVERAGE = 0.9995
# 이 값은 문장을 이루는 문자 중 센텐스피스가 학습 데이터에서 학습하려고 하는 문자의 비율입니다.
# 예를 들어 0.9995는 전체 문자 중 99.95%의 문자를 학습하려고 한다는 뜻이며,
# 나머지 매우 드문 문자는 무시됩니다.


In [8]:
def ensure_spm_model(texts: List[str]):
    # sentencepiece 모델을 새로 학습시키는 함수입니다.
    # 매개변수 texts: 모델을 학습할 때 사용하는 텍스트 문장들의 리스트입니다.
    # 예: ["안녕하세요.", "저는 학생입니다"]와 같은 형태입니다.

    # 기존에 학습되어 있던 센텐스피스 모델, vocab 파일을 삭제합니다.
    # 새로 학습된 모델을 만들기 위해 기존 파일을 항상 지우고 다시 생성합니다.
    for p in [f"{SPM_MODEL_PREFIX}.model", f"{SPM_MODEL_PREFIX}.vocab"]:
        if Path(p).exists():      # 파일이 존재하면
            os.remove(p)          # 파일을 삭제합니다. 예: "spm_ko_nopunct.model" 삭제

    # 임시로 사용할 텍스트 파일 이름을 정의합니다.
    tmp = "spm_corpus_nopunct.txt"

    # texts 리스트에 있는 모든 텍스트를 깨끗하게 정리한 후 임시 텍스트 파일에 저장합니다.
    with open(tmp, "w", encoding="utf-8") as f:
        for t in texts:
            # t는 texts 리스트의 각 문장에 대해 반복합니다.
            # clean_text(t)는 특수문자를 제거하고, replace("\n", " ")로 줄바꿈 문자를 공백으로 바꿉니다.
            t = clean_text(t).replace("\n", " ")
            f.write(t + "\n")     # 예: "안녕\n잘 지내니?" → "안녕 잘 지내니"로 변경된 후 파일에 저장됩니다.

    # SentencePieceTrainer로 sentencepiece 모델을 학습시킵니다.
    spm.SentencePieceTrainer.Train(
        input=tmp,                       # 위에서 만든 임시 텍스트 파일을 학습 데이터로 사용합니다.
        model_prefix=SPM_MODEL_PREFIX,   # 생성될 모델의 파일 이름 앞부분입니다. 예: "spm_ko_nopunct"
        vocab_size=SPM_VOCAB_SIZE,       # 생성할 단어 조각(토큰)의 최대 개수입니다. 예: 8000개
        model_type="unigram",            # 토큰을 만드는 방법을 지정합니다. 여기서는 unigram 방식을 사용합니다.
        character_coverage=CHAR_COVERAGE,# 전체 문자 중 몇 %를 포함할지 비율을 설정합니다. 예: 0.9995 (99.95%)
        unk_id=0,                        # <unk> (알 수 없는 단어) 토큰을 사용하지 않으므로 0으로 설정합니다.
        bos_id=-1,                       # <bos> (begin of sentence, 문장 시작) 토큰을 사용하지 않으므로 -1로 설정합니다.
        eos_id=-1,                       # <eos> (end of sentence, 문장 끝) 토큰을 별도로 설정하지 않으므로 -1로 설정합니다.
        pad_id=-1,                       # <pad> 토큰을 사용하지 않습니다.
        remove_extra_whitespaces=True    # 연속 공백을 하나로 줄입니다. 예: "안녕   잘   지내" → "안녕 잘 지내"
    )

    # 학습이 끝난 후 더 이상 필요 없는 임시 텍스트 파일을 삭제합니다.
    os.remove(tmp)


In [9]:
class SpmVocab:
    # 주어진 텍스트 데이터를 이용해 문장을 숫자로 변환하거나 숫자를 문장으로 바꾸어 주는 클래스입니다.

    def __init__(self, texts: List[str], model_file: str = SPM_MODEL):
        # 클래스를 초기화 메서드입니다.
        # texts: 센텐스피스 모델을 학습시킬 때 사용하는 텍스트 데이터의 리스트입니다.
        # 예: ["안녕하세요.", "저는 인공지능입니다."]
        # model_file: 미리 학습된 센텐스피스 모델 파일 이름입니다. 기본값으로는 SPM_MODEL을 사용합니다.

        # 주어진 texts 데이터로 센텐스피스 모델을 학습시킵니다.
        ensure_spm_model(texts)

        # SentencePieceProcessor 객체를 생성합니다. 이 객체를 이용해 문장-숫자 변환을 합니다.
        self.sp = spm.SentencePieceProcessor()

        # 학습된 모델을 로드(불러오기)합니다.
        # 로드한 모델을 사용하여 문장을 숫자로 변환하거나 숫자를 문장으로 변환할 수 있습니다.
        self.sp.Load(model_file)

        # 특수한 용도로 미리 지정된 토큰(단어조각) 4개가 있기 때문에,
        # 실제 센텐스피스에서 나오는 숫자 앞에 4를 더해서 충돌을 피합니다.
        # 예: 센텐스피스가 만든 토큰 번호가 0이면, 실제 사용 시엔 앞에 4를 더해서 4가 됩니다.
        self.offset = 4

        # 센텐스피스가 생성한 토큰의 개수(어휘의 크기)입니다.
        self.spm_size = self.sp.GetPieceSize()

        # 전체 어휘 크기입니다. 센텐스피스 토큰 수에 미리 정해진 특수토큰 4개를 더한 값입니다.
        self.size = self.offset + self.spm_size

        # 특수 토큰의 번호를 설정합니다.
        # pad_idx: 길이 맞추기 위한 빈 칸
        # sos_idx: 문장 시작
        # eos_idx: 문장 끝
        # unk_idx: 알 수 없는 단어
        self.pad_idx, self.sos_idx, self.eos_idx, self.unk_idx = 0, 1, 2, 3

        def encode(self, text: str, add_sos=True, add_eos=True, max_len: int | None = None) -> List[int]:
            # 입력된 텍스트 문장을 숫자 리스트로 바꿔주는 함수입니다.
            # text: 숫자로 변환할 텍스트 문장입니다. 예: "안녕하세요"
            # add_sos: 문장 시작 토큰(<sos>)을 리스트 맨 앞에 추가할지 여부입니다. 기본값 True.
            # add_eos: 문장 끝 토큰(<eos>)을 리스트 맨 뒤에 추가할지 여부입니다. 기본값 True.
            # max_len: 숫자 리스트의 최대 길이. 초과하면 잘라냅니다.
        
            # 텍스트를 정리합니다.
            text = clean_text(text)
        
            # 센텐스피스를 이용해 텍스트를 숫자로 변환합니다.
            # 각 숫자에 self.offset(4)을 더해 특수 토큰 번호와 충돌하지 않게 합니다.
            ids = [i + self.offset for i in self.sp.EncodeAsIds(text)]
        
            # 최대 길이 제한이 있다면, sos/eos까지 고려한 실제 허용 길이를 계산해서 문장 숫자를 자릅니다.
            if max_len is not None:
                budget = max_len - (1 if add_sos else 0) - (1 if add_eos else 0)
                budget = max(0, budget)
                ids = ids[:budget]
        
            # <sos> 추가
            if add_sos:
                ids = [self.sos_idx] + ids
        
            # <eos> 추가
            if add_eos:
                ids = ids + [self.eos_idx]
        
            # 최종 숫자 리스트 반환
            return ids

    def decode(self, ids: List[int]) -> str:
        # 숫자 리스트를 다시 사람이 읽을 수 있는 텍스트 문장으로 바꿔주는 함수입니다.
        # ids: 숫자 리스트입니다. 예: [1, 16, 38, 2]
    
        # 리스트에서 offset(4)을 뺀 숫자를 센텐스피스 모델을 통해 다시 문자로 바꿉니다.
        # 특수 토큰 번호는 무시하고 실제 센텐스피스 토큰 번호만 변환합니다.
        toks = [i - self.offset for i in ids if i >= self.offset]
    
        try:
            # 센텐스피스 모델의 DecodeIds를 사용해 숫자 리스트를 텍스트로 변환합니다.
            # 예: [12, 34] → "안녕"
            return self.sp.DecodeIds(toks)
        except Exception:
            # DecodeIds에서 문제가 생기면 각 숫자를 직접 문자 조각으로 변환해서 이어 붙입니다.
            # IdToPiece는 숫자를 작은 문자 조각(piece)으로 바꿔줍니다.
            return "".join(self.sp.IdToPiece(i) for i in toks)


In [10]:
class SpmVocab:
    # 주어진 텍스트 데이터를 이용해 문장을 숫자로 변환하거나 숫자를 문장으로 바꾸어 주는 클래스입니다.

    def __init__(self, texts: List[str], model_file: str = SPM_MODEL):
        # 클래스를 초기화 메서드입니다.
        # texts: 센텐스피스 모델을 학습시킬 때 사용하는 텍스트 데이터의 리스트입니다.
        # 예: ["안녕하세요.", "저는 학생입니다"]
        # model_file: 미리 학습된 센텐스피스 모델 파일 이름입니다. 기본값으로는 SPM_MODEL을 사용합니다.

        # 주어진 texts 데이터로 센텐스피스 모델을 학습시킵니다.
        ensure_spm_model(texts)

        # SentencePieceProcessor 객체를 생성합니다. 이 객체를 이용해 문장-숫자 변환을 합니다.
        self.sp = spm.SentencePieceProcessor()

        # 학습된 모델을 로드(불러오기)합니다.
        # 로드한 모델을 사용하여 문장을 숫자로 변환하거나 숫자를 문장으로 변환할 수 있습니다.
        self.sp.Load(model_file)

        # 특수한 용도로 미리 지정된 토큰(단어조각) 4개가 있기 때문에,
        # 실제 센텐스피스에서 나오는 숫자 앞에 4를 더해서 충돌을 피합니다.
        # 예: 센텐스피스가 만든 토큰 번호가 0이면, 실제 사용 시엔 앞에 4를 더해서 4가 됩니다.
        self.offset = 4

        # 센텐스피스가 생성한 토큰의 개수(어휘의 크기)입니다.
        self.spm_size = self.sp.GetPieceSize()

        # 전체 어휘 크기입니다. 센텐스피스 토큰 수에 미리 정해진 특수토큰 4개를 더한 값입니다.
        self.size = self.offset + self.spm_size

        # 특수 토큰의 번호를 설정합니다.
        # pad_idx(0): 길이 맞추기 위한 빈 칸
        # sos_idx(1): 문장 시작
        # eos_idx(2): 문장 끝
        # unk_idx(3): 알 수 없는 단어
        self.pad_idx, self.sos_idx, self.eos_idx, self.unk_idx = 0, 1, 2, 3

    def encode(
        self,
        text: str,
        add_sos: bool = True,
        add_eos: bool = True,
        max_len: int | None = None,
    ) -> List[int]:
        # 입력된 텍스트 문장을 숫자 리스트로 바꿔주는 함수입니다.
        # text: 숫자로 변환할 텍스트 문장입니다. 예: "안녕하세요"
        # add_sos: 문장 시작 토큰(<sos>)을 숫자 리스트 맨 앞에 추가할지 여부입니다. 기본값 True.
        # add_eos: 문장 끝 토큰(<eos>)을 숫자 리스트 맨 뒤에 추가할지 여부입니다. 기본값 True.
        # max_len: 숫자 리스트의 최대 길이. 지정되면 초과된 부분은 잘라냅니다.

        # 텍스트 정리
        text = clean_text(text)

        # 센텐스피스 토크나이저로 문장을 숫자로 변환하고 offset을 더해 충돌을 방지합니다.
        ids = [i + self.offset for i in self.sp.EncodeAsIds(text)]

        # 최대 길이가 지정된 경우, sos/eos 추가를 고려한 실제 허용 길이를 계산해 문장을 잘라냅니다.
        if max_len is not None:
            budget = max_len - (1 if add_sos else 0) - (1 if add_eos else 0)
            budget = max(0, budget)
            ids = ids[:budget]

        # sos 추가
        if add_sos:
            ids = [self.sos_idx] + ids

        # eos 추가
        if add_eos:
            ids = ids + [self.eos_idx]

        # 최종 ID 리스트 반환
        return ids

    def decode(self, ids: List[int]) -> str:
        # 숫자 리스트를 다시 사람이 읽을 수 있는 텍스트 문장으로 바꿔주는 함수입니다.
        # ids: 숫자 리스트입니다. 예: [1, 16, 38, 2]

        # 리스트에서 offset(4)을 뺀 숫자를 센텐스피스 모델을 통해 다시 문자로 바꿉니다.
        # 특수 토큰 번호는 무시하고 실제 센텐스피스 토큰 번호만 변환합니다.
        toks = [i - self.offset for i in ids if i >= self.offset]

        try:
            # 센텐스피스 모델의 DecodeIds를 사용해 숫자 리스트를 텍스트로 변환합니다.
            # 예: [12, 34] → "안녕"
            return self.sp.DecodeIds(toks)
        except Exception:
            # DecodeIds에서 문제가 생기면(오류 발생 시) 각 숫자를 직접 문자조각으로 변환해서 붙여줍니다.
            # IdToPiece는 숫자를 작은 문자 조각으로 바꿔줍니다.
            return "".join(self.sp.IdToPiece(i) for i in toks)


# 센텐스피스 모델을 이용해 vocab(어휘 사전)을 생성합니다.
# vocab은 문장을 숫자 형태로 바꾸거나 숫자를 다시 문장으로 변환할 때 사용됩니다.
# 예: "안녕" → [1, 200, 342, 2]
vocab = SpmVocab(texts=list(df["Q"].values) + list(df["A"].values))

# vocab의 전체 크기와 센텐스피스 모델에서 생성한 토큰(작은 단어 조각)의 개수를 출력합니다.
# 예: "[SPM] vocab size: 8004 (SPM pieces: 8000)" 라는 결과가 나오면,
# vocab 전체 크기는 8004개이며 그중 실제 토큰은 8000개라는 뜻입니다.
print(f"[SPM] vocab size: {vocab.size} (SPM pieces: {vocab.spm_size})")




[SPM] vocab size: 8004 (SPM pieces: 8000)


In [11]:
# 입력 문장(질문)의 최대 길이를 설정합니다.
# 입력된 문장이 이 길이를 초과하면 뒷부분을 잘라냅니다.
# 예: 최대 길이가 64일 때, 70 단어의 질문은 뒤의 6단어가 잘립니다.
MAX_SRC_LEN = 64

# 출력 문장(답변)의 최대 길이를 설정합니다.
# 출력된 문장이 이 길이를 초과하면 뒷부분을 잘라냅니다.
# 예: 최대 길이가 64일 때, 70 단어의 답변은 뒤의 6단어가 잘립니다.
MAX_TRG_LEN = 64

# 입력 문장을 숫자로 바꿀 때 무작위로 일부 단어를 '<unk>'(알 수 없는 단어)로 바꾸는 확률입니다.
# 예: "안녕 오늘 날씨 어때?"라는 문장을 숫자로 바꿀 때 0.05(5%) 확률로 일부 단어가 '<unk>'가 될 수 있습니다.
SRC_TOKEN_DROPOUT_P = 0.05

In [12]:
# 아래는 하나의 데이터 예시를 표현하는 데이터 클래스입니다.
# 하나의 데이터 예시는 입력(src)과 출력(trg) 숫자 리스트로 구성됩니다.
@dataclass
class Example:
    # 입력 문장을 숫자로 표현한 리스트입니다.
    # 예: 질문이 "안녕 오늘 날씨 어때"라면 숫자 리스트는 [1, 100, 45, 32, 2] 와 같이 됩니다.
    # 여기서 1은 <sos>(문장 시작), 2는 <eos>(문장 끝)을 의미합니다.
    src: List[int]

    # 출력 문장을 숫자로 표현한 리스트입니다.
    # 예: 답변이 "오늘은 맑아"라면 숫자 리스트는 [1, 45, 28, 2] 와 같이 됩니다.
    # 마찬가지로 1은 문장 시작(<sos>), 2는 문장 끝(<eos>)입니다.
    trg: List[int]


class QADataset(Dataset):
    # QADataset 클래스는 질문(Q)과 대답(A) 데이터를
    # 인공지능이 학습할 수 있는 숫자 형태로 바꾸어주는 역할을 합니다.
    # Dataset은 학습용 PyTorch DataLoader에서 쉽게 사용할 수 있습니다.
    # 즉, 이 클래스는 데이터를 ‘묶음(batch)’ 형태로 만들어주는 준비 단계입니다.

    def __init__(self, df: pd.DataFrame, vocab: SpmVocab):
        # QADataset 클래스의 생성자입니다.

        # self.vocab은 클래스 내부에서 vocab을 사용하기 위해 저장하는 변수입니다.
        self.vocab = vocab

        # self.samples: 질문-대답 쌍을 숫자 리스트 형태로 저장하는 리스트
        self.samples: List[Example] = []

        # df의 각 row를 가져와 숫자 리스트로 변환
        for _, row in df.iterrows():

            # 질문(Q)을 숫자 리스트로 변환
            src_ids = vocab.encode(
                row["Q"],
                add_sos=True,
                add_eos=True,
                max_len=MAX_SRC_LEN
            )

            # 대답(A)을 숫자 리스트로 변환
            trg_ids = vocab.encode(
                row["A"],
                add_sos=True,
                add_eos=True,
                max_len=MAX_TRG_LEN
            )

            # Example 객체로 묶어서 samples에 저장
            self.samples.append(Example(src=src_ids, trg=trg_ids))

    def __len__(self):
        # 전체 데이터 개수를 반환
        return len(self.samples)

    def __getitem__(self, i):
        # i번째 Example을 반환
        return self.samples[i]


def collate_fn(batch: List[Example], pad_idx: int = 0):
    # collate_fn: 여러 개의 Example을 묶어서 텐서로 변환해 batch 형태로 만드는 함수

    def pad_to_max(seqs: List[List[int]]):
        # 가장 긴 길이에 맞춰 padding 수행
        mx = max(len(s) for s in seqs)
        return [s + [pad_idx] * (mx - len(s)) for s in seqs]

    # 질문(src) padding
    src = pad_to_max([b.src for b in batch])

    # 대답(trg) padding
    trg = pad_to_max([b.trg for b in batch])

    # 텐서 변환
    src = torch.tensor(src, dtype=torch.long)
    trg = torch.tensor(trg, dtype=torch.long)

    # trg_input: 마지막 토큰 제거
    trg_input = trg[:, :-1]

    # trg_output: 첫 토큰 제거
    trg_output = trg[:, 1:]

    # pad mask 생성
    src_key_pad = (src == pad_idx)
    trg_key_pad = (trg_input == pad_idx)

    # 텐서 반환
    return src, src_key_pad, trg_input, trg_output, trg_key_pad


# train/valid split
train_df, valid_df = train_test_split(df, test_size=0.15, random_state=42)

# train_df 표시
train_df


,Q,A,label
299,귀가 가려워,누가 욕하고 있나봐요,0
2742,시트지로 리폼해야겠다,직접 하면 더 애착이 갈 거 같아요,0
1090,느낌이 좋아,사랑의 느낌이길 바라요,0
1194,대놓고 질린대,잠시 거리를 두고 생각해보세요,0
7955,저주스럽다 모든게 다,불행한 생각은 덜 하는게 당신에게 좋아요,1
...,...,...,...
11284,좋아했지만 고백은 못하겠어,애틋한 사랑이네요,2
5191,화장실,화장실 가세요,0
5390,6개월이 지나도 왜이런거죠,물리적 시간에 비례하지 않으니까요,1
860,내가 제일 문제인 듯,당신은 하나밖에 없는 소중한 사람이에요,0


In [13]:
valid_df

,Q,A,label
8169,죽을거 같네,나쁜 생각 하지 마세요,1
900,내일 시험이야,컨디션 조절 하세요,0
8075,정말내 자신이 싫다,자신은 사랑해주세요,1
7625,이별후 네달째,바쁘게 살면서 잊어가요,1
2816,쌍커풀 해볼까,눈은 기본이죠,0
...,...,...,...
11133,좋아하는 사람이 쓰레기면 어떻게 할꺼야,쓰레기 옆에 있기 싫어서 도망갈 거예요,2
4914,피곤해서 하루종일 누워있는 중,충전하는 시간 그 자체로 소중합니다,0
4417,차 살까,판단은 당신 몫이에요,0
777,남편이 하나도 안 도와줘,돕는 게 아니라 같이 하는 거예요,0


In [14]:
# reset_index 함수를 사용해서 데이터의 인덱스를 다시 0부터 시작하게 재정렬합니다.
# drop=True는 기존의 인덱스를 제거하고, inplace=True는 실제 데이터프레임에 바로 적용한다는 뜻입니다.
# 예를 들어 인덱스가 [1,4,7,9]였다면 재정렬 후엔 [0,1,2,3]으로 바뀌게 됩니다.
train_df.reset_index(drop=True, inplace=True)

train_df   # 재정렬된 인덱스를 확인할 수 있습니다.


,Q,A,label
0,귀가 가려워,누가 욕하고 있나봐요,0
1,시트지로 리폼해야겠다,직접 하면 더 애착이 갈 거 같아요,0
2,느낌이 좋아,사랑의 느낌이길 바라요,0
3,대놓고 질린대,잠시 거리를 두고 생각해보세요,0
4,저주스럽다 모든게 다,불행한 생각은 덜 하는게 당신에게 좋아요,1
...,...,...,...
10044,좋아했지만 고백은 못하겠어,애틋한 사랑이네요,2
10045,화장실,화장실 가세요,0
10046,6개월이 지나도 왜이런거죠,물리적 시간에 비례하지 않으니까요,1
10047,내가 제일 문제인 듯,당신은 하나밖에 없는 소중한 사람이에요,0


In [15]:
valid_df.reset_index(drop=True, inplace=True)

valid_df   # 검증 데이터도 마찬가지로 재정렬된 인덱스를 확인할 수 있습니다.


,Q,A,label
0,죽을거 같네,나쁜 생각 하지 마세요,1
1,내일 시험이야,컨디션 조절 하세요,0
2,정말내 자신이 싫다,자신은 사랑해주세요,1
3,이별후 네달째,바쁘게 살면서 잊어가요,1
4,쌍커풀 해볼까,눈은 기본이죠,0
...,...,...,...
1769,좋아하는 사람이 쓰레기면 어떻게 할꺼야,쓰레기 옆에 있기 싫어서 도망갈 거예요,2
1770,피곤해서 하루종일 누워있는 중,충전하는 시간 그 자체로 소중합니다,0
1771,차 살까,판단은 당신 몫이에요,0
1772,남편이 하나도 안 도와줘,돕는 게 아니라 같이 하는 거예요,0


In [16]:
# 이제 실제 인공지능 모델이 학습할 수 있는 형태의 데이터셋을 만듭니다.
# train_ds는 학습용 데이터셋입니다. QADataset 클래스를 이용해 만들고, 숫자로 변환된 질문과 대답 데이터를 담고 있습니다.
# 예: "오늘 날씨 어때?" → [1,23,45,2] 와 같은 숫자 형태로 바뀌어 저장됩니다.
train_ds = QADataset(train_df, vocab)
train_ds   # 학습용 데이터셋을 화면에서 확인할 수 있습니다.


In [17]:
# valid_ds는 검증용 데이터셋으로 같은 방식으로 만듭니다. 검증용 데이터는 학습이 아니라 모델의 성능을 평가하는 용도로 쓰입니다.
valid_ds = QADataset(valid_df, vocab)

valid_ds   # 검증용 데이터셋을 화면에서 확인할 수 있습니다.


In [18]:
# vocab.pad_idx는 문장의 길이를 맞추기 위한 빈칸(<pad>)의 숫자값입니다. 여기서는 0입니다.
# 예를 들어 [1,23,2]라는 문장을 길이 5로 맞추려면 뒤에 [0,0]을 붙여 [1,23,2,0,0]으로 만듭니다.
vocab.pad_idx


0

In [19]:
# DataLoader는 데이터를 묶음(batch)으로 나누고, 학습 시 편하게 데이터를 사용할 수 있게 해주는 역할을 합니다.
# train_loader는 학습용 데이터를 묶음 형태로 제공합니다.
# batch_size=BATCH_SIZE는 묶음의 크기입니다(예: 64개의 데이터를 한번에 묶어서 제공).
# shuffle=True는 데이터를 섞어서 매번 다른 순서로 제공합니다. 그래야 모델이 데이터를 골고루 학습할 수 있습니다.
# collate_fn은 데이터를 묶어서 패딩을 추가하고 최종적으로 텐서 형태로 만듭니다.
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=lambda b: collate_fn(b, vocab.pad_idx))


In [20]:
# valid_loader는 검증용 데이터를 묶음 형태로 제공합니다.
# shuffle=False로 검증용 데이터는 섞지 않고 매번 같은 순서로 제공합니다(성능 평가를 일관되게 하려는 목적입니다).
valid_loader = DataLoader(
    valid_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=lambda b: collate_fn(b, vocab.pad_idx)
)


class LSTMSeq2Seq(nn.Module):
    def __init__(self, vocab_size: int, d_model=256,
                 num_encoder_layers=3, num_decoder_layers=3,
                 dropout=0.2, pad_idx=0):
        super().__init__()

        self.pad_idx = pad_idx
        self.d_model = d_model
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers

        # === 여기 수정 ===
        # src / tgt 임베딩 이름을 src_embed / tgt_embed 로 통일
        self.src_embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.tgt_embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)

        self.encoder = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model,
            num_layers=num_encoder_layers,
            dropout=dropout if num_encoder_layers > 1 else 0.0,
            bidirectional=False,
        )

        self.dec_cells = nn.ModuleList(
            [nn.LSTMCell(d_model, d_model) for _ in range(num_decoder_layers)]
        )

        self.dropout = nn.Dropout(dropout)
        self.generator = nn.Linear(d_model, vocab_size, bias=False)

        # weight tying
        self.generator.weight = self.tgt_embed.weight



    def encode(self, src: torch.Tensor, src_key_padding_mask: torch.Tensor):
        src = src.transpose(0, 1)
        emb = self.dropout(self.src_embed(src))

        lengths = (~src_key_padding_mask).sum(dim=1).clamp(min=1).cpu()
        packed = pack_padded_sequence(emb, lengths, enforce_sorted=False)

        _, (h_n, c_n) = self.encoder(packed)

        h0 = h_n[-1]
        c0 = c_n[-1]
        return h0, c0


    def decode(self, tgt: torch.Tensor, memory, tgt_key_padding_mask: torch.Tensor):
        h0, c0 = memory
        tgt = tgt.transpose(0, 1)   # (B, T) -> (T, B)
        T, B = tgt.size(0), tgt.size(1)

        h_states, c_states = [], []
        for l in range(self.num_decoder_layers):
            if l == 0:
                h_states.append(h0)
                c_states.append(c0)
            else:
                h_states.append(torch.zeros(B, self.d_model, device=tgt.device))
                c_states.append(torch.zeros(B, self.d_model, device=tgt.device))

        logits_steps = []

        for t in range(T):
            y_t = tgt[t]  # (B,)
            # === 여기서도 tgt_embed 사용 ===
            emb = self.dropout(self.tgt_embed(y_t))

            step_mask = None
            if tgt_key_padding_mask is not None:
                step_mask = (~tgt_key_padding_mask[:, t]).unsqueeze(1)

            x = emb

            # 디코더 LSTMCell 스택 처리
            for l, cell in enumerate(self.dec_cells):
                h_prev, c_prev = h_states[l], c_states[l]
                nh, nc = cell(x, (h_prev, c_prev))

                if step_mask is not None:
                    h_states[l] = torch.where(step_mask, nh, h_prev)
                    c_states[l] = torch.where(step_mask, nc, c_prev)
                else:
                    h_states[l], c_states[l] = nh, nc

                x = self.dropout(h_states[l])

            h_top = x
            logits_t = self.generator(h_top)
            logits_steps.append(logits_t)

        logits = torch.stack(logits_steps, dim=0)
        return logits


    def forward(self, src, src_key_padding_mask, tgt_input, tgt_key_padding_mask):
        memory = self.encode(src, src_key_padding_mask)
        logits = self.decode(tgt_input, memory, tgt_key_padding_mask)
        return logits


In [21]:
# LSTMSeq2Seq 모델의 인스턴스를 생성(초기화)합니다.
# 이 모델은 주어진 질문을 입력으로 받아서 적절한 답변을 출력하는 역할을 합니다.
model = LSTMSeq2Seq(
        vocab_size=vocab.size,               # 전체 사용 가능한 단어(토큰)의 개수입니다. 예: 8000개
        d_model=256,                         # 각 단어가 숫자 벡터로 표현될 때의 벡터 길이(차원)입니다. 예: 256
        num_encoder_layers=3,                # 인코더의 LSTM층을 몇 개 쌓을 것인지 설정합니다. 예: 3개 층
        num_decoder_layers=3,                # 디코더의 LSTMCell 층 개수를 설정합니다. 예: 3개 층
        dropout=0.2,                         # 드롭아웃 비율로, 일부 뉴런을 무작위로 끄는 비율입니다. 예: 20%
        pad_idx=vocab.pad_idx                # 패딩(빈칸)의 숫자 인덱스입니다. 일반적으로 0으로 설정됩니다.
).to(DEVICE)

# ls_ce_loss는 손실 함수(loss function)로, 인공지능 모델이 얼마나 잘못 예측했는지 숫자로 표현합니다.
# 손실 값이 낮을수록 인공지능 모델이 잘 예측하고 있는 것입니다.
def ls_ce_loss(logits: torch.Tensor, targets: torch.Tensor, ignore_index: int, label_smoothing: float = 0.1):
    # 매개변수 설명:
    # logits: 인공지능이 예측한 각 단어의 점수(확률)입니다. 크기는 (T,B,V) → (단어 길이, 배치 크기, 전체 단어 수)입니다.
    # targets: 실제 정답 문장의 단어들입니다. 크기는 (B,T) → (배치 크기, 단어 길이)입니다.
    # ignore_index: 손실 계산 시 무시할 값(일반적으로 padding 값인 0)을 의미합니다.
    # label_smoothing: 정답을 살짝 부드럽게(smoothed) 하여 모델이 정답만을 과신하지 않게 돕습니다(기본값 0.1).

    logits = logits.transpose(0,1).contiguous()     # 크기를 (B,T,V) 형태로 변경합니다.
    V = logits.size(-1)                              # 전체 단어의 수(V)를 가져옵니다.
    logits = logits.view(-1, V)                      # logits을 2차원으로 평평하게 만들어 계산을 쉽게 합니다.
                                                     # 예: (B×T,V), 전체 단어 확률을 평평하게 만듭니다.
    targets = targets.reshape(-1)                    # 정답 단어들도 평평한 형태로 바꿉니다(B×T,).

    # F.cross_entropy 함수가 label_smoothing을 지원하는지 확인합니다.
    sig = inspect.signature(F.cross_entropy)
    if "label_smoothing" in sig.parameters:
        # cross_entropy 함수로 예측값(logits)과 정답(targets) 간의 손실을 계산합니다.
        return F.cross_entropy(
            logits,
            targets,
            ignore_index=ignore_index,
            label_smoothing=label_smoothing
        )

    # 위의 방식이 안될 경우(fallback) 직접 계산합니다.
    # log_softmax는 logits를 확률 값으로 바꿉니다.
    log_probs = F.log_softmax(logits, dim=-1)
    
    # 실제 정답 분포(true_dist)를 만듭니다. 전체 확률을 부드럽게(smoothing) 만듭니다.
    with torch.no_grad():
        # 전체 값에 아주 작은 확률(label_smoothing / (V - 1))을 줍니다.
        true_dist = torch.zeros_like(log_probs)
        true_dist.fill_(label_smoothing / (V - 1))
    
        # 정답 위치는 높은 확률(1 - label_smoothing)을 줍니다.
        mask = (targets != ignore_index)   # 패딩 부분은 제외하고 계산합니다.
        idx = targets[mask]
        true_dist[mask, idx] = 1.0 - label_smoothing
    
    # 실제 분포(true_dist)와 예측 분포(log_probs) 차이를 계산하여 손실을 만듭니다.
    loss = -(true_dist * log_probs).sum(dim=-1)
    
    # ignore_index(예: 패딩)가 아닌 값들만 손실을 계산하고 평균을 냅니다.
    loss = loss[targets != ignore_index].mean()
    
    # 계산된 최종 손실 값을 반환합니다.
    return loss


In [22]:
class NoamOpt:
    # NoamOpt 클래스는 인공지능 모델이 학습할 때 사용하는 옵티마이저(optimizer)를 조절하는 역할을 합니다.
    # 이 클래스는 "학습률 스케줄러(learning rate scheduler)"라고도 부릅니다.
    # 학습률(learning rate)이란 인공지능이 한 번 학습할 때마다 얼마나 빠르게 배우는지 결정하는 숫자입니다.
    # 이 숫자가 너무 크면 학습이 불안정하고, 너무 작으면 학습이 너무 느려집니다.
    # 이 클래스는 적절한 학습률을 자동으로 조절하여 안정적으로 학습할 수 있도록 도와줍니다.

    # NoamOpt 학습률 계산 공식:
    # 학습률 = factor × d_model^-0.5 × min(step^-0.5, step × warmup^-1.5)
    # 이 공식은 처음에는 작은 학습률로 천천히 학습을 시작하고,
    # 점점 빠르게 올라갔다가 특정 지점을 이후 다시 천천히 내려가게 만듭니다.

    def __init__(self, optimizer, d_model, warmup_steps=4000, factor=1.0, min_lr=1e-6):
        # 클래스의 초기화 함수입니다. 이 함수는 클래스가 처음 만들어질 때 실행됩니다.
        #
        # 매개변수 설명:
        # optimizer: 인공지능이 학습할 때 사용하는 기본 옵티마이저입니다(예: AdamW).
        # d_model: 단어를 숫자 벡터로 표현한 벡터의 차원입니다(예: 256).
        # warmup_steps: 학습률이 처음에 서서히 올라가는 단계(steps)의 수입니다. 기본값은 4000입니다.
        # factor: 학습률 전체 크기를 조정하는 숫자입니다(기본값은 1.0이며, 학습 속도를 빠르게 또는 느리게 만듭니다).
        # min_lr: 학습률의 최소값으로, 학습률이 이 값 이하로 내려가지 않습니다(기본값은 0.000001).

        self.optimizer = optimizer         # 실제 사용하는 옵티마이저입니다.
        self.d_model = d_model             # 벡터의 차원(d_model)을 저장합니다.
        self.warmup = warmup_steps         # warmup 단계를 저장합니다.
        self.factor = factor               # 학습률 크기를 조정하는 factor를 저장합니다.
        self.min_lr = min_lr               # 최소 학습률을 저장합니다.

        self._step = 0                     # 현재 학습한 횟수를 저장하는 변수입니다(초기값 0).
        self._rate = 0.0                   # 현재 학습률을 저장합니다(초기값 0.0).

    def step(self):
        # step 함수는 한 번 학습할 때마다 자동으로 호출됩니다.
        # 이 함수는 매번 학습률을 새로 계산해서 옵티마이저에 전달합니다.

        self._step += 1                       # 학습을 진행할 때마다 step을 하나씩 증가시킵니다.
        rate = self.rate()                    # 현재 학습률을 계산합니다.

        # 옵티마이저가 사용하는 모든 파라미터 그룹에 새 학습률(rate)을 적용합니다.
        # 학습률은 최소값(min_lr)보다 작아지지 않도록 유지합니다.
        for p in self.optimizer.param_groups:
            p['lr'] = max(self.min_lr, rate)

        self._rate = rate                     # 현재 사용된 학습률을 저장합니다.
        self.optimizer.step()                 # 실제 옵티마이저의 step 함수를 호출하여 학습을 진행합니다.

    def zero_grad(self):
        # zero_grad 함수는 다음번 학습을 시작하기 전에 인공지능 모델의 기울기(gradient)를 0으로 초기화합니다.
        # 기울기를 초기화하는 매 학습마다 새롭게 기울기를 계산하기 위해 필수입니다.

        self.optimizer.zero_grad()            # 실제 옵티마이저의 기울기를 모두 0으로 초기화합니다.

    def rate(self, step=None):
        # rate 함수는 주어진 학습 단계(step)에 따라 학습률을 계산하여 반환합니다.
        # 이 함수는 학습률 스케줄러 공식을 따라 계산합니다.
        # 매개변수 step: 학습률을 계산할 때 사용할 단계(steps)입니다.
        #   → step 값이 따로 주어지지 않으면, 현재의 self._step 값을 사용합니다.

        if step is None:
            step = self._step

        # 학습률을 공식에 따라 계산합니다.
        calculated_rate = (
            self.factor
            * (self.d_model ** -0.5)
            * min(step ** -0.5, step * (self.warmup ** -1.5))
        )

        # 예를 들어 d_model이 256이고, warmup_steps가 4000이며, 현재 step이 2000일 때,
        # 계산된 학습률은 factor × (256^(-0.5) × min(2000^(-0.5), 2000 × 4000^(-1.5)) 와 같이 계산됩니다.

        return calculated_rate                # 계산된 학습률을 반환합니다.


# -----------------------------------------------------------
# 🔥 n-gram 반복 방지 함수 (주석 온전하게 유지)
# -----------------------------------------------------------

def ngram_ban(tokens: list[int], n: int) -> set[int]:
    # ngram_ban 함수는 인공지능 모델이 문장을 생성할 때 같은 단어가 반복되는 현상을 방지하는 역할을 합니다.
    # 이 현상을 “n-gram 반복 방지”라고 합니다.
    # 여기서 n-gram은 ‘연속 n개의 단어’를 의미합니다.
    # 예를 들어, 3-gram("나는 밥을 먹었다")이면 ["나는", "밥을", "먹었다"]처럼 3개 단어가 묶인 의미입니다.

    # 매개변수 설명:
    # tokens: 지금까지 생성된 문장(단어들)을 숫자 형태로 표현한 리스트입니다.
    #        예: [1, 5, 7, 10, 7, 12]
    # n: n-gram의 크기. 몇 개의 연속된 단어가 반복되지 않도록 막을지를 결정합니다.
    #    예: n=3 이라면, 같은 3개 단어 묶음이 연달아 나타나지 않도록 하는 기능입니다.

    # n이 2보다 작거나 문장의 길이가 n-1보다 짧으면 반복을 검사할 필요가 없으므로 빈 set(금지어)를 반환합니다.
    if n < 2 or len(tokens) < n - 1:
        return set()

    table = {}   # 반복되는 단어 묶음을 기록할 테이블(딕셔너리)입니다.

    # 문장을 앞에서부터 한 칸씩 옮겨가면서 (n-1)개 단어로 이루어진 묶음(prefix)을 만듭니다.
    # 같은 prefix에 바로 뒤에 오는 단어(next)를 기록합니다.
    for i in range(len(tokens) - n + 1):
        prefix = tuple(tokens[i : i + n - 1])  # 현재 위치에서 n-1개의 단어 묶음을 만듭니다.
        nxt = tokens[i + n - 1]                # 이 묶음 뒤에 오는 바로 다음 단어를 가져옵니다.

        # 만약 prefix가 table에 없다면 새로운 빈 집합을 넣습니다.
        # 이미 있으면 set에 새로운 다음 단어(nxt)를 추가해줍니다.
        table.setdefault(prefix, set()).add(nxt)

    # 예: tokens=[1,5,7,10,5,7,12], n=3일 때,
    #     prefix (1,5) 다음에 7이 나타났으면 table[(1,5)]={7}이 됩니다.
    #     prefix (5,7) 뒤에 10이 오고 12가 나타났으면 table[(5,7)]={10,12}가 됩니다.

    # 이제 문장의 마지막 (n-1)개 단어를 prefix로 사용해서
    # 해당 prefix 뒤에 등장하면 안 되는(이미 반복되었던) 단어 목록을 찾습니다.
    # 이렇게 반환된 단어들은 금지해야 하는 단어들로, 모델의 생성 제한 용도로 쓰입니다.
    pref = tuple(tokens[len(tokens) - (n - 1) :])

    # prefix 뒤에 이미 등장했던 단어가 있으면 단어 집합을 반환합니다.
    # 없으면 빈 set 반환.
    return table.get(pref, set())

    # 예: tokens=[1,5,7,10,5,7,10], n=3일 때,
    # 마지막 prefix=(5,7) 뒤에 10이 여러 번 등장했으므로 {10}을 반환합니다.
    # 즉, 다음 단어로 10이 다시 나오면 안 되므로 그걸 방지하는 용도로 사용됩니다.

    @torch.no_grad()
    def reply(model: TransformerSeq2Seq, text: str, max_len: int = 64):
        # reply 함수는 학습된 인공지능 모델이 사용자의 입력 문장에 적절한 답변을 만들어 주는 기능을 합니다.
        # 이 함수는 실제 서비스에서 사용자가 챗봇에게 질문하면, 챗봇이 답변을 만들어주는 역할을 합니다.
        #
        # 매개변수 설명:
        # model: 답변을 만드는 데 사용할 학습된 인공지능 모델입니다.
        # text: 사용자 입력 문장(질문)입니다. 예: "오늘 날씨 어때?"
        # max_len: 생성할 답변 문장의 최대 길이입니다. 기본값은 64단어입니다.
        #
        # @torch.no_grad()는 이 함수 내부에서는 기울기(gradient)를 계산하지 않고,
        # 오직 문장을 만드는 출력용(inference) 작업만 하도록 설정하는 것입니다.
        # 실제 서비스에서 문장을 만들 때만 사용하는 기능이며, 학습할 때는 사용하지 않습니다.
    
        # generate_beam 함수는 "빔 서치(beam search)"라는 방법을 이용해서 좋은 문장을 만듭니다.
        # 빔 서치란, 단어를 하나씩 예측해가면서 여러 가지 후보 문장을 동시에 가장 좋은 문장을 찾는 방법입니다.
        # beam_size=4는 한 번에 4가지 문장 후보를 기억하면서 탐색한다는 뜻입니다.
        # max_len=64는 생성할 문장의 최대 길이이며, min_len=6는 최소 길이를 의미합니다.
        # alpha=0.6은 문장 길이에 따른 점수를 조절하는 비율이며, 길이를 0.6으로 설정하여 길이 적절한 문장을 생성하게 합니다.
        # no_repeat_ngram_size=3은 3개의 연속된 단어가 똑같이 다시 반복되는 것을 방지하는 기능이고,
        # repetition_penalty=1.2는 같은 단어가 자꾸 반복되는 것을 줄이기 위한 기능입니다.
    
        ids = generate_beam(
            model,
            text,
            beam_size=4,
            max_len=max_len,
            min_len=6,
            alpha=0.6,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2
        )
    
        # 생성된 문장(ids)에서 처음에 있는 시작 토큰(<sos>)을 제거하고,
        # 끝에 있는 문장 끝(<eos>) 이후의 단어들도 제거합니다.
        # 모델이 "답변의 시작"을 표시하기 위해 <sos>를 넣기 때문입니다.
        # 예를 들어, ids가 [1, 34, 67, 23, 2, 89,10], 10이 <sos>, 2가 <eos>라면,
        # 1 제거 후: [34, 67, 23, 2, 89,10]
        # 2(eos)가 나오면 그 이후는 의미 없으므로 잘라냅니다.
    
        # 첫 단어가 <sos> 토큰이면 제거
        if ids and ids[0] == vocab.sos_idx:
            ids = ids[1:]
    
        # 문장 끝 토큰(<eos>)이 있으면 그 뒤를 자른다
        if vocab.eos_idx in ids:
            ids = ids[: ids.index(vocab.eos_idx)]
    
        # 숫자로 이루어진 문장(ids)을 실제 사람이 읽을 수 있는 텍스트 문장으로 변환합니다.
        # 예: [34, 67, 23] → "날씨 맑아요"
        return vocab.decode(ids)      # 최종 생성된 문장을 반환합니다.

    def run_epoch(model, loader, optimizer: NoamOpt | None,
                  label_smoothing=0.1, src_token_dropout=0.05, grad_clip=1.0):
        # run_epoch 함수는 인공지능 모델에 데이터를 이용해 한 번(epoch) 학습하거나 성능을 평가할 때 사용하는 함수입니다.
        # 이 함수는 학습용 데이터(트레이닝 데이터)나 검증용 데이터(밸리데이션 데이터)를 한 번 전체 순회하는 역할을 합니다.
        #
        # 매개변수 설명:
        # model: 인공지능 모델입니다(LSTMSeq2Seq 모델).
        # loader: DataLoader로 묶은 데이터를 묶음(batch)들 순서대로 제공합니다.
        # optimizer: 모델의 학습을 관리는 옵티마이저 객체입니다.
        #            optimizer가 None이면 모델을 평가(검증)만 하고 학습은 하지 않습니다.
        # label_smoothing: 손실(loss)을 계산할 때 정답의 확률 분포를 부드럽게 만들어
        #                  모델이 정답을 과신하지 않게 하는 정규화 역할입니다.
        #                  기본값은 0.1입니다.
        # src_token_dropout: 입력 데이터(src)의 일부 단어를 무작위로 보이지 않는 단어(<unk>)로 바꾸어
        #                    모델이 더 강인하게 학습하게 합니다.
        #                    기본값은 0.05(5%)입니다.
        # grad_clip: 기울기(gradient)를 일정 크기로 제한하여 학습이 폭발적으로 증가하거나
        #            불안정해지는 것을 막아줍니다.
        #            기본값은 1.0입니다.
    
        is_train = optimizer is not None        # optimizer가 있으면 훈련(train)모드, 없으면 평가(eval) 모드입니다.
        model.train(is_train)                   # 모델을 훈련 모드 또는 평가 모드로 설정합니다.
    
        total, n = 0.0, 0                       # 손실(loss)의 합(total)과 데이터 개수(n)를 저장하는 변수입니다.

        @torch.no_grad()
        def reply(model: TransformerSeq2Seq, text: str, max_len: int = 64):
            # reply 함수는 학습된 인공지능 모델이 사용자의 입력 문장에 적절한 답변을 만들어 주는 기능을 합니다.
            # 이 함수는 실제 서비스에서 사용자가 챗봇에 질문하면, 챗봇이 답변을 만들어주는 역할을 합니다.
            #
            # 매개변수 설명:
            # model: 답변을 만드는 데 사용할 학습된 인공지능 모델입니다.
            # text: 사용자가 입력한 문장(질문)입니다. 예: "오늘 날씨 어때?"
            # max_len: 생성할 답변 문장의 최대 길이입니다. 기본값은 64단어입니다.
        
            # @torch.no_grad()는 이 함수 내부에서는 기울기(gradient)를 계산하지 않고,
            # 오직 문장을 만드는 추론(inference) 작업만 하도록 설정하는 것입니다.
            # 실제 서비스에서는 문장을 만들 때만 사용하는 기능이며, 학습할 때는 사용하지 않습니다.
        
            # generate_beam 함수는 '빔 서치(beam search)'라는 방법을 이용해서 좋은 문장을 만듭니다.
            # 빔 서치란, 단어를 하나씩 예측할 때 여러 개의 후보 문장을 동시에 평가해서
            # 가장 좋은 문장을 찾는 방법입니다.
            # beam_size=4는 한번에 최대 4가지 문장 후보를 기억하면서 탐색한다는 뜻입니다.
            # max_len=64는 생성할 문장의 최대 길이이고, min_len=6은 최소 길이를 의미합니다.
            # alpha는 문장 길이에 대한 패널티이며, 기본값 0.6으로 설정되어 길이 적절한 문장을 생성하게 합니다.
            # no_repeat_ngram_size=3은 3개 단어가 연달아 반복되지 않도록 막아주는 기능입니다.
            # repetition_penalty=1.2는 같은 단어가 자꾸 반복되는 것을 줄이기 위한 기능입니다.
        
            ids = generate_beam(
                model, text,
                beam_size=4,
                max_len=max_len,
                min_len=6,
                alpha=0.6,
                no_repeat_ngram_size=3,
                repetition_penalty=1.2
            )
        
            # 생성된 문장(ids)에서 처음에 있는 시작 토큰(<sos>)을 제거하고,
            # 문장 끝 토큰(<eos>)이 있으면 단어들을 잘라냅니다.
            # 이렇게 처리하는 이유는 <sos>, <eos>는 사람이 쓰지 않기 때문입니다.
            # 예를 들어, ids가 [1, 34, 67, 23, 2, 89, 10], 1이 <sos>, 2가 <eos>라면,
            # 정답 ids는 [34, 67, 23]로 줄어듭니다.
        
            if ids and ids[0] == vocab.sos_idx:    # 첫 단어가 시작 토큰(<sos>)인지 확인합니다.
                ids = ids[1:]                      # 시작 토큰을 제거합니다.
        
            if vocab.eos_idx in ids:               # 문장 끝 토큰(<eos>)이 있다면 그 위치를 찾습니다.
                ids = ids[:ids.index(vocab.eos_idx)]  # 문장 끝 토큰 뒤의 단어들을 제거합니다.
        
            # 숫자로 이루어진 문장(ids)을 실제 사람이 읽을 수 있는 텍스트 문장으로 변환합니다.
            return vocab.decode(ids)               # 최종 생성된 문장을 반환합니다.


In [23]:
def run_epoch(model, loader, optimizer: NoamOpt | None,
              label_smoothing=0.1, src_token_dropout=0.05, grad_clip=1.0):
    # run_epoch 함수는 인공지능 모델이 데이터를 이용해 한 번(1 epoch) 학습하거나
    # 성능을 평가할 때 사용하는 함수입니다.
    # 이 함수는 학습용 데이터(트레이닝 데이터)나 검증용 데이터(밸리데이션 데이터)를
    # 한 번 전체 순회하는 역할을 합니다.

    # 매개변수 설명:
    # model: 인공지능 모델입니다(LSTMSeq2Seq 모델).
    # loader: DataLoader로 묶은 데이터 묶음(batch)을 순서대로 제공합니다.
    # optimizer: 모델의 학습을 관리하는 도구입니다.
    #            NoamOpt일 수도 있고, None일 수도 있습니다.
    #            optimizer가 None이면 모델을 평가(검증)만 하고 학습은 하지 않습니다.
    # label_smoothing: 손실(loss)을 계산할 때 정답의 확률 분포를 부드럽게 만들어
    #                  모델이 정답을 과신하지 않게 하는 정도입니다.
    #                  기본값은 0.1입니다.
    # src_token_dropout: 입력 데이터(src)의 일부 단어를 무작위로 알 수 없는 단어(<unk>)로 바꿔
    #                    모델이 더 강인하게 학습하게 합니다.
    #                    기본값은 0.05(5%)입니다.
    # grad_clip: 기울기(gradient)를 일정 크기로 제한하여
    #            학습이 폭발적으로 증가하거나 불안정해지는 것을 막아줍니다.
    #            기본값은 1.0입니다.

    is_train = optimizer is not None       # optimizer가 있으면 훈련(train)모드, 없으면 평가(eval)모드입니다.
    model.train(is_train)                  # 모델을 훈련 모드 또는 평가 모드로 설정합니다.

    total, n = 0.0, 0                      # 손실(loss)의 합(total)과 데이터 개수(n)를 저장하는 변수입니다.
   
    # loader에서 한 번에 한 묶음(batch)의 데이터를 가져와서 반복합니다.
    for (src, src_key_pad, trg_input, trg_output, trg_key_pad) in loader:

        # 각 데이터를 GPU나 CPU 장치로 이동시킵니다.
        src, src_key_pad = src.to(DEVICE), src_key_pad.to(DEVICE)
        trg_input, trg_output, trg_key_pad = (
            trg_input.to(DEVICE), trg_output.to(DEVICE), trg_key_pad.to(DEVICE)
        )

        # 훈련 시에만 입력(src) 문장의 일부 단어를 랜덤하게 알 수 없는 단어(<unk>)로 바꿉니다.
        if is_train and src_token_dropout > 0:
            # 랜덤한 값이 dropout 비율보다 작으면 해당 위치의 단어를 바꿉니다.
            drop = (torch.rand_like(src, dtype=torch.float) < src_token_dropout)

            # 패딩과 같은 특수 단어(<pad>, <sos>, <eos>)는 바꾸지 않고 유지합니다.
            special = (
                (src == vocab.pad_idx)
                | (src == vocab.sos_idx)
                | (src == vocab.eos_idx)
            )

            # 랜덤으로 선택된 일반 단어 위치를 <unk>(알 수 없는 단어)로 바꿉니다.
            src = torch.where(drop & (~special),
                              torch.full_like(src, vocab.unk_idx),
                              src)
            # 예를 들어 src가 [1, 23, 56, 2]이고, 56이 dropout 대상이면
            # [1, 23, 3(<unk>), 2]로 바뀝니다.

        # 모델에 데이터를 넣고 예측한 결과(logits)를 얻습니다.
        # logits: 각 단어별 예측 확률로, 크기는 (T,B,V)입니다.
        logits = model(src, src_key_pad, trg_input, trg_key_pad)

        # 실제 정답(trg_output)과 모델의 예측(logits)을 비교하여 손실(loss)을 계산합니다.
        loss = ls_ce_loss(logits, trg_output,
                          ignore_index=vocab.pad_idx,
                          label_smoothing=label_smoothing)
        
        # 학습 모드일 때는 실제로 학습을 진행합니다.
        if is_train:
            optimizer.zero_grad()                       # 이전 학습 때 남은 기울기(gradient)를 0으로 초기화합니다.
            loss.backward()                             # 손실을 줄이는 방향으로 기울기를 계산합니다.
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)  # 기울기가 너무 커지지 않도록 제한합니다.
            optimizer.step()                            # 계산된 기울기를 이용해 실제 모델 파라미터(가중치)를 업데이트(학습)합니다.

        bs = src.size(0)                                # 현재 묶음(batch)의 크기입니다.
        total += float(loss.item()) * bs                # 현재 묶음의 손실을 총 손실에 추가합니다.
        n += bs                                         # 현재 묶음의 개수를 전체 개수에 추가합니다.

    # 최종적으로 평균 손실을 계산해서 반환합니다.
    # 손실값이 낮을수록 모델이 데이터를 잘 학습한 것입니다.
    return total / max(1, n)



In [24]:
def generate_beam(
    model: LSTMSeq2Seq,
    text: str,
    beam_size=4,
    max_len=64,
    min_len=6,
    alpha=0.6,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2
) -> List[int]:
    # generate_beam 함수는 사용자가 입력한 문장에서 인공지능 모델이 가장 적절한 답변 문장을 생성하는 함수입니다.
    # 이 때 "빔 서치(beam search)"라는 방법을 사용하여 여러 후보 문장을 비교해서 가장 좋은 문장을 선택하는 방식입니다.

    # 매개변수 설명:
    # model: 답변 생성에 사용할 인공지능 모델입니다(LSTMSeq2Seq).
    # text: 사용자가 입력한 문장입니다. 예: "오늘 날씨 어때?"
    # beam_size: 매번 가장 좋은 문장을 선택할 때, 몇 가지 후보 문장을 유지할지를 정하는 값입니다(기본값: 4).
    # max_len: 생성할 수 있는 답변 문장의 최대 길이입니다(기본값: 64단어).
    # min_len: 답변 문장의 최소 길이입니다. 최소 길이가 짧아지면 문장 끝(<eos>) 토큰을 생성하지 않도록 합니다(기본값: 6).
    # alpha: 문장 길이에 따라 점수를 보정하는 길이 보너스(length penalty)를 결정하는 값입니다(기본값: 0.6).
    # no_repeat_ngram_size: 같은 단어가 연속으로 반복되는 것을 방지하기 위한 단어 묶음 크기입니다(기본값: 3).
    # repetition_penalty: 같은 단어가 반복될 때 점수를 깎는 비율입니다(기본값: 1.2).

    # 입력한 문장(text)을 인공지능 모델이 이해할 수 있는 숫자 형태로 변환합니다.
    # add_sos=True는 문장 시작 토큰(<sos>)을 추가, add_eos=True는 문장 끝 토큰(<eos>)을 추가하는 옵션입니다.
    src_ids = vocab.encode(text, add_sos=True, add_eos=True, max_len=MAX_SRC_LEN)

    # 숫자 리스트 형태의 문장을 텐서(Tensor) 형태로 변환하고 GPU(또는 CPU) 장치에 올립니다.
    src = torch.tensor([src_ids], dtype=torch.long, device=DEVICE)

    # 패딩(<pad>) 위치를 표시합니다. 여기서는 입력 문장(src) 중에서 pad_idx(일반적으로 0) 위치를 찾아 표시합니다.
    src_key_pad = (src == vocab.pad_idx)

    # 입력 문장을 인코더에 넣어서 문장의 의미를 담은 벡터(memory)를 얻습니다.
    memory = model.encode(src, src_key_pad)

    # beams 리스트는 문장을 생성하면서 유지할 후보 문장들입니다.
    # 각 후보는 (지금까지 생성된 단어 리스트, 문장의 점수(logprob), 문장이 끝났는지 여부)로 이루어집니다.
    beams = [([vocab.sos_idx], 0.0, False)]   # 처음에는 <sos> 토큰 하나 있는 문장으로 시작합니다.

    # 단어를 하나씩 생성하면서 최대 길이(max_len)까지 반복합니다.
    for step in range(max_len):

        new_beams = []  
        # 이번 단계에서 새롭게 생성될 후보 문장들입니다.

        curr_tgts = [b[0] for b in beams]   
        # 현재 후보 문장들을 가져옵니다.

        lens = [len(t) for t in curr_tgts]  
        # 각 후보 문장의 길이를 저장합니다.

        T = max(lens)  
        # 가장 긴 문장 길이를 찾습니다.

        # 모든 후보 문장을 같은 길이로 맞추기 위해 짧은 문장 뒤에 패딩(<pad>)을 추가합니다.
        tgt = []
        tgt_keypad = []

        for t in curr_tgts:
            padded = t + [vocab.pad_idx] * (T - len(t))
            tgt.append(padded)
            tgt_keypad.append(([False] * len(t)) + ([True] * (T - len(t))))

        # 후보 문장들을 텐서로 변환합니다.
        tgt = torch.tensor(tgt, dtype=torch.long, device=DEVICE)
        tgt_keypad = torch.tensor(tgt_keypad, dtype=torch.bool, device=DEVICE)

        B = len(beams)  
        # 현재 후보 문장의 개수입니다.

        # 인코더의 출력을 후보 문장 수만큼 복사하여 사용합니다.
        h0, c0 = memory
        h0_rep = h0.expand(B, h0.size(1)).contiguous()
        c0_rep = c0.expand(B, c0.size(1)).contiguous()
        mem_rep = (h0_rep, c0_rep)

        # 디코더를 사용하여 다음 단어의 점수(logits)를 얻습니다.
        logits = model.decode(tgt, mem_rep, tgt_keypad)

        last_logits = logits[-1]  
        # 마지막으로 생성된 단어의 점수를 가져옵니다.

        # 각 후보 문장에서 다음 단어를 선택합니다.
        for i, (tokens, logp, ended) in enumerate(beams):

            # ended=True 이면 이미 문장이 종료된 상태 → 그대로 유지합니다.
            if ended:
                new_beams.append((tokens, logp, True))
                continue

            scores = last_logits[i]    # 현재 후보 문장의 다음 단어 점수(logit)

            # 패딩 토큰 <pad> 는 절대 선택되지 않도록 매우 낮은 점수로 설정합니다.
            scores[vocab.pad_idx] = -1e9

            # 시작 토큰 <sos> 도 출력에 나오면 안 되므로 선택 불가로 설정합니다.
            scores[vocab.sos_idx] = -1e9

            # 최소 길이에 도달하기 전에는 <eos> 토큰을 선택하지 못하도록 패널티 적용합니다.
            if len(tokens) <= min_len:
                scores[vocab.eos_idx] = -1e9

            # 반복되는 단어를 방지하기 위해 repetition_penalty 를 적용해 점수를 보정합니다.
            if repetition_penalty and repetition_penalty != 1.0:
                seen = set(tokens)
                for tid in list(seen):
                    if scores[tid] > 0:
                        scores[tid] = scores[tid] / repetition_penalty
                    else:
                        scores[tid] = scores[tid] * repetition_penalty

            # 같은 n-gram 반복을 막기 위해 ngram_ban 함수로 금지어를 얻고 점수를 패널티 처리합니다.
            if no_repeat_ngram_size and no_repeat_ngram_size >= 2:
                bans = ngram_ban(tokens, no_repeat_ngram_size)
                for tid in bans:
                    scores[tid] = -1e9


            # 다음 단어 확률 계산 → 가장 높은 단어 top-k 개 선택(beam_size 개)
            log_probs = F.log_softmax(scores, dim=-1)
            topk = torch.topk(log_probs, beam_size)

            # top-k 후보들을 새로운 beam 후보로 저장합니다.
            for k in range(beam_size):
                tid = int(topk.indices[k])     # 다음 단어 id
                logp_next = float(topk.values[k])
                new_tokens = tokens + [tid]

                # EOS 가 나오면 ended 상태로 전환합니다.
                ended_flag = (tid == vocab.eos_idx)

                new_beams.append((new_tokens, logp + logp_next, ended_flag))


        # 문장 길이가 길어질수록 패널티(length penalty)를 적용해 최종 후보 점수를 조정합니다.
        def length_penalty(L, alpha=0.6):
            return ((5 + L) ** alpha) / ((5 + 1) ** alpha)

        scored = []
        for (ts, lp, ed) in new_beams:
            score = lp / length_penalty(len(ts), alpha)
            scored.append((score, ts, lp, ed))

        # 점수 순으로 정렬하여 상위 beam_size 개만 다음 단계로 유지합니다.
        scored.sort(key=lambda x: x[0], reverse=True)
        beams = [(ts, lp, ed) for (_, ts, lp, ed) in scored[:beam_size]]

        # 모든 beam 이 종료되었으면 탐색 중단합니다.
        if all(ed for (_, _, ed) in beams):
            break

    # 최종적으로 가장 좋은 점수를 가진 문장을 선택해 반환합니다.
    best = max(beams, key=lambda b: b[1] / length_penalty(len(b[0]), alpha))
    return best[0]   # 최종 선택된 문장의 단어 id 리스트


In [25]:
@torch.no_grad()
def reply(model: LSTMSeq2Seq, text: str, max_len: int = 64):
    """
    학습된 LSTM Seq2Seq 모델로 답변 한 문장을 생성하는 함수.
    """
    model.eval()

    # beam search로 토큰 id 시퀀스 생성
    ids = generate_beam(
        model, text,
        beam_size=4,
        max_len=max_len,
        min_len=6,
        alpha=0.6,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
    )

    # 맨 앞 <sos> 제거
    if ids and ids[0] == vocab.sos_idx:
        ids = ids[1:]

    # 중간에 <eos> 있으면 거기까지만 사용
    if vocab.eos_idx in ids:
        eos_pos = ids.index(vocab.eos_idx)
        ids = ids[:eos_pos]

    # id 시퀀스를 텍스트로 디코딩
    return vocab.decode(ids)

In [26]:
# 아래는 인공지능 모델을 학습할 때 사용하는 중요한 설정값(하이퍼파라미터)들을 정의합니다.
# 이 값들은 모델 학습이 어떻게 진행될지 세부적으로 결정해주는 역할을 합니다.

epochs = 200
# epochs(에폭)는 전체 데이터를 몇 번 반복해서 학습할지 결정하는 값입니다.
# 예를 들어, epochs=200은 전체 데이터를 200번 학습하는 것을 의미합니다.

base_lr = 1.0
# base_lr(기본 학습률)은 모델이 학습할 때 사용하는 기본적인 학습률을 나타냅니다.
# 이 값이 너무 크면 학습이 불안정하고, 너무 작으면 학습이 느리게 진행됩니다.
# 기본값은 1.0을 사용하며, 이후 NoamOpt라는 학습률 조정을 사용해서 자동으로 조정됩니다.

warmup_steps = 4000
# warmup_steps는 학습률을 천천히 높이는 단계의 수입니다.
# 초반에 낮은 학습률에서 시작해 이 단계까지 점점 학습률이 올라갑니다.
# 예: warmup_steps=4000은 처음 4000번의 학습까지는 점점 더 큰 학습률을 사용하며,
# 이후로는 다시 줄어듭니다.

label_smoothing = 0.1
# label_smoothing은 정답 레이블(정답 단어 확률)을 부드럽게 만들어서 모델이 특정 단어만 확신하지 않도록 합니다.
# 예를 들어, label_smoothing=0.1이면 정답 확률을 90%로 낮추고, 나머지 확률을 다른 단어들에 나누어줍니다.

src_token_dropout = 0.05
# src_token_dropout은 입력 문장(src)의 일부 단어를 랜덤하게 <unk>(모르는 단어)로 바꾸는 비율입니다.
# 예: 5% 확률로 ["오늘", "날씨", "어때"] → ["오늘", "<unk>", "어때"] 처럼 바뀌게 됩니다.
# 이렇게 하면 모델이 더 다양한 상황에 잘 대응하게 됩니다.

weight_decay = 1e-4
# weight_decay는 모델의 가중치 값들이 너무 커지는 것을 막아 과적합(overfitting)을 방지하는 데 사용됩니다.
# 일반적으로 아주 작은 값을 사용합니다(여기서는 0.0001).

early_stop_patience = 10
# early_stop_patience는 검증 데이터로 측정한 성능이 계속 개선되지 않을 때, 학습을 일찍 멈추는 기준입니다.
# 예: patience가 10이면, 10번의 학습(epoch) 동안 성능이 전혀 좋아지지 않으면 학습을 종료합니다.

save_path = "seq2seq_lstm.pt"
# save_path는 학습된 모델을 저장할 파일의 경로(이름)입니다.
# 학습이 끝나거나 중간에 좋은 성능이 나왔을 때 이 경로로 모델이 저장됩니다.

# 현재 학습과 검증 데이터의 개수, 단어 사전(vocab)의 크기를 화면에 출력해서 확인합니다.
print(f"Train size: {len(train_ds)} / Valid size: {len(valid_ds)} / Vocab size (total): {vocab.size} (SPM pieces: {vocab.spm_size})")



Train size: 10049 / Valid size: 1774 / Vocab size (total): 8004 (SPM pieces: 8000)


In [27]:
# 실제로 학습을 진행할 때 사용하는 옵티마이저(optimizer)와 학습률 조정기(scheduler)를 설정합니다.
# 옵티마이저는 모델이 더 나은 결과를 내도록 모델의 가중치(파라미터)를 조금씩 조정해주는 역할입니다.
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.0,                    # 시작 학습률은 NoamOpt가 관리하므로 0.0으로 둡니다.
    weight_decay=weight_decay,
    betas=(0.9, 0.98),
    eps=1e-9
)

# NoamOpt는 학습을 진행하면서 자동으로 학습률을 조절하는 역할을 합니다.
sched = NoamOpt(
    optimizer,
    d_model=256,               # 단어 벡터의 차원(d_model)입니다.
    warmup_steps=warmup_steps,
    factor=base_lr,
    min_lr=1e-6
)

# best_val은 가장 좋은 검증(validation) 손실 값을 저장하는 변수입니다.
best_val = float("inf")
no_improve = 0   # 성능이 좋아지지 않은 횟수를 저장합니다.


# epochs 수만큼 반복하여 학습을 진행합니다.
for ep in range(epochs):

    # 학습 데이터를 사용해 모델을 한 번 학습시키고 손실을 계산합니다.
    train_loss = run_epoch(
        model, train_loader, optimizer=sched,
        label_smoothing=label_smoothing,
        src_token_dropout=src_token_dropout
    )

    # 검증 데이터를 사용해 모델의 성능을 평가하고 손실을 계산합니다.
    val_loss = run_epoch(
        model, valid_loader, optimizer=None,
        label_smoothing=label_smoothing,
        src_token_dropout=0.0
    )

    # 학습과 검증 손실을 출력하여 학습 상태를 확인합니다.
    print("=" * 100)
    print(f"[ep:{ep+1:04d}/{epochs}] lr={sched._rate:.6g}  train_loss={train_loss:.4f}  valid_loss={val_loss:.4f}")

    # 매 10번의 학습마다 실제 문장을 넣어보고 모델이 어떤 답변을 만드는지 직접 확인합니다.
    if (ep + 1) % 10 == 0:
        tests = [
            "정상적으로 안돌아가 싶다",
            "너무 힘든데 어떻게 하지",
            "가스불을 켜놓고 나온거 같아",
        ]

        print("\n[예시 문장 테스트]")
        for t in tests:
            print(f"{t:16} -> {reply(model, t)}\n")

    # 검증 손실이 이전보다 좋아지면 모델을 저장하고, no_improve를 초기화합니다.
    if val_loss < best_val - 1e-4:
        best_val = val_loss
        no_improve = 0

        # 모델의 가중치와 설정들을 파일로 저장합니다.
        torch.save({
            'model': model.state_dict(),
            'spm_model': SPM_MODEL,
            'vocab_size': vocab.size,
            'config': {
                'd_model': 256,
                'enc_layers': 3,
                'dec_layers': 3,
                'dropout': 0.2,
                'SPM_VOCAB_SIZE': SPM_VOCAB_SIZE,
                'arch': 'lstm_seq2seq_noamfit'
            }
        }, save_path)

    else:
        # 좋아지지 않았으면 증가합니다.
        no_improve += 1

        # 좋아지지 않은 횟수가 early_stop_patience를 넘으면 학습을 일찍 종료합니다.
        if no_improve >= early_stop_patience:
            print(
                f"[EARLY STOP] 성능이 {early_stop_patience}번 동안 좋아지지 않아 학습을 조기 종료합니다. "
                f"가장 좋은 valid_loss={best_val:.4f}"
            )
            break


[ep:0001/200] lr=3.90344e-05  train_loss=8.8200  valid_loss=8.1650
[ep:0002/200] lr=7.80687e-05  train_loss=8.0021  valid_loss=7.7461
[ep:0003/200] lr=0.000117103  train_loss=7.7581  valid_loss=7.5059
[ep:0004/200] lr=0.000156137  train_loss=7.5039  valid_loss=7.2149
[ep:0005/200] lr=0.000195172  train_loss=7.2266  valid_loss=6.9298
[ep:0006/200] lr=0.000234206  train_loss=6.9541  valid_loss=6.6844
[ep:0007/200] lr=0.000273241  train_loss=6.7021  valid_loss=6.4606
[ep:0008/200] lr=0.000312275  train_loss=6.4757  valid_loss=6.2747
[ep:0009/200] lr=0.000351309  train_loss=6.2846  valid_loss=6.1138
[ep:0010/200] lr=0.000390344  train_loss=6.1114  valid_loss=5.9866

[예시 문장 테스트]
정상적으로 안돌아가 싶다    -> 사랑은 항상죠

너무 힘든데 어떻게 하지    -> 그게 사람을 주세요

가스불을 켜놓고 나온거 같아  -> 그게 항상죠

[ep:0011/200] lr=0.000429378  train_loss=5.9564  valid_loss=5.8583
[ep:0012/200] lr=0.000468412  train_loss=5.8112  valid_loss=5.7682
[ep:0013/200] lr=0.000507447  train_loss=5.6768  valid_loss=5.6757
[ep:0014/200] lr=0.00054648

In [28]:
# 아래의 tests 리스트는 인공지능 모델이 실제로 얼마나 잘 동작하는지
# 확인하기 위해 만든 예시 질문 문장들입니다.
# 인공지능 모델에게 이 문장들을 입력하면,
# 모델은 각 질문에 맞는 적절한 답변을 생성합니다.
tests = [
    "3박4일 놀러가고 싶다",
    "SD카드 안돼",
    "가스를 켜놓고 나온거 같아",
    "지망 학교 떨어졌어",
    "SNS보면 나만 빼고 다 행복해보여",
]

# 예를 들어, 사용자가 "3박4일 놀러가고 싶다"고 입력하면,
# 인공지능 모델이 "여행 다녀오면 기분이 좋아질 거예요!" 같은 답변을 생성합니다.

# 생성된 질문 문장으로 실제 테스트를 실행합니다.
print("\n[샘플 테스트]\n")

# tests 리스트에 있는 각 문장을 순서대로 하나씩 모델에 입력하여 답변을 생성합니다.
for t in tests:
    # 사용자의 질문(Q)을 화면에 출력합니다.
    print(f"Q: {t}")

    # reply 함수는 인공지능 모델을 이용하여 질문 문장(t)에 대한 답변을 생성하는 함수입니다.
    # 예를 들어 reply(model, "SD카드 안돼")를 실행하면
    # "카드가 인식이 안되나요? 다시 꽂아보세요."와 같은 답변이 생성됩니다.
    answer = reply(model, t)

    # 인공지능 모델이 생성한 답변(A)을 화면에 출력합니다.
    print(f"A: {answer}\n")



[샘플 테스트]

Q: 3박4일 놀러가고 싶다
A: 여행은 언제나 좋죠

Q: SD카드 안돼
A: 다시 새로 사는 게 마음 편해요

Q: 가스를 켜놓고 나온거 같아
A: 빨리 집에 돌아가서 끄고 나오세요

Q: 지망 학교 떨어졌어
A: 즐거운 시간이 될 거 같아요

Q: SNS보면 나만 빼고 다 행복해보여
A: 술 안 마셔도 놀 수 있어요



In [ ]:
while True:  
    # 무한히 반복하여 사용자의 입력을 계속 기다립니다.
    try:
        # input 함수는 사용자가 입력한 문장을 받아오는 함수입니다.
        # 사용자는 화면에 나타나는 > 뒤에 문장을 입력합니다.
        u = input("> ").strip()
        # 입력받은 문장에서 앞뒤 공백을 제거합니다(.strip()).
        # 예를 들어 사용자가 "  안녕? "이라고 입력하면 "안녕?"으로 바뀝니다.

    except EOFError:
        # EOFError는 입력이 갑자기 종료될 때 발생합니다.
        # 이 오류가 발생하면 입력이 끝난 것으로 무한반복을 종료(break)합니다.
        break

    if not u:
        # 사용자가 아무 것도 입력하지 않고 그냥 엔터만 누르면, 빈 문자열이 입력됩니다.
        # 이 경우에는 답변을 만들지 않고, 다시 입력을 기다립니다(continue).
        continue

    if u.lower() in ("/quit", "/exit"):
        # 사용자가 "/quit" 또는 "/exit"를 입력하면,
        # 대화형 테스트를 종료합니다.
        # 예를 들어 "/quit"를 입력하면 프로그램이 종료됩니다.
        break

    # 사용자가 정상적으로 질문 문장을 입력한 경우(reply 함수 사용):
    # reply 함수는 입력 문장(u)을 인공지능 모델(model)에 넣어 답변을 만들어줍니다.
    # 예를 들어 사용자가 "기분 우울해"라고 입력하면,
    # 모델이 "힘들었겠어요. 어디 아픈 곳은 없나요?"와 같은 답변을 생성할 수 있습니다.
    # max_len은 생성되는 답변의 최대 길이를 80단어로 제한합니다.
    answer = reply(model, u, max_len=80)

    # 생성된 답변(answer)을 화면에 출력합니다.
    print(answer)
